# GPU_Test

This is a notebook to test the GPU functionality. It uses the MNIST data and quickly runs a small CNN. Each epoch should take just seconds to finish.

This is taken from an excellent article I used to help me get the GPU environment up and running. I have slightly modified it by removing the tensorboard parts and adding the graphing. This is, however, not anything we haven't done in class or the labs, so I don't feel it inappropriate to use his code.

[How to Install TensorFlow with GPU Support on Windows 10 (Without Installing CUDA) UPDATED!](https://www.pugetsystems.com/labs/hpc/How-to-Install-TensorFlow-with-GPU-Support-on-Windows-10-Without-Installing-CUDA-UPDATED-1419/)

In [1]:
import time
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.backend.tensorflow_backend import set_session
#magic
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

Using TensorFlow backend.


In [2]:
mnist = tf.keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# reshape and rescale data for the CNN

train_images = train_images.reshape(60000, 28, 28, 1)

test_images = test_images.reshape(10000, 28, 28, 1)

train_images, test_images = train_images/255, test_images/255

In [3]:
# This bit of code allows keras/tf to dynamically grow the GPU memory. This may or may not be solving the GPU
# issue I was having. Taken from https://www.cicoria.com/keras-tensor-flow-cublas_status_not_initialized/
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
start_time=time.time()

history = model.fit(train_images, train_labels, batch_size=128, epochs=15, verbose=1,
         validation_data=(test_images, test_labels))

print('Training took {} seconds'.format(time.time()-start_time))

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
33024/60000 [===============>..............] - ETA: 46s - loss: 0.3504 - acc: 0.8926

In [ ]:
# Check out our train loss and test loss over epochs.
train_loss = history.history['loss']
test_loss = history.history['val_loss']

# Set figure size.
plt.figure(figsize=(12, 8))

# Generate line plot of training, testing loss over epochs.
plt.plot(train_loss, label='Training Loss', color='#185fad')
plt.plot(test_loss, label='Testing Loss', color='orange')

# Set title
plt.title('Training and Testing Loss by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Sparse Categorical Crossentropy', fontsize = 18)
plt.xticks(range(15))

plt.legend(fontsize = 18);

In [ ]:
# Get the predictions for X_test_new
predictions = model.predict_classes(test_images)
print(f'Accuracy score: {metrics.accuracy_score(test_labels, predictions)}')

In [ ]:
cf = pd.DataFrame(metrics.confusion_matrix(test_labels, predictions))
cf